In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
%matplotlib inline
#import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss

datadir = 'input/'
gatrain = pd.read_csv(os.path.join(datadir,'gender_age_train.csv'),
                      index_col='device_id')
gatest = pd.read_csv(os.path.join(datadir,'gender_age_test.csv'),
                     index_col = 'device_id')
phone = pd.read_csv(os.path.join(datadir,'phone_brand_device_model.csv'))
# Get rid of duplicate device ids in phone
phone = phone.drop_duplicates('device_id',keep='first').set_index('device_id')

gatrain['row'] = np.arange(gatrain.shape[0])
gatest['row'] = np.arange(gatest.shape[0])

brandencoder = LabelEncoder().fit(phone.phone_brand)
phone['brand'] = brandencoder.transform(phone['phone_brand'])
gatrain['brand'] = phone['brand']
gatest['brand'] = phone['brand']

m = phone.phone_brand.str.cat(phone.device_model)
modelencoder = LabelEncoder().fit(m)
phone['model'] = modelencoder.transform(m)
gatrain['model'] = phone['model']
gatest['model'] = phone['model']

gatrain['to_group'] = gatrain.gender.astype(str)+'.'+gatrain.age.astype(str)+'.'+gatrain.brand.astype(str)+'.'+gatrain.model.astype(str)

gatrain['to_group_bm'] = gatrain.brand.astype(str)+'.'+gatrain.model.astype(str)
gatest['to_group_bm'] = gatest.brand.astype(str)+'.'+gatest.model.astype(str)

In [6]:
tr_groupby_bm = gatrain.groupby('to_group_bm')
te_groupby_bm = gatest.groupby('to_group_bm')

In [2]:
gatrain.head(2)

,gender,age,group,row,brand,model,to_group,to_group_bm
device_id,,,,,,,,
-8076087639492063270,M,35,M32-38,0,51,843,M.35.51.843,51.843
-2897161552818060146,M,35,M32-38,1,51,843,M.35.51.843,51.843


In [3]:
gatest.head(2)

,row,brand,model,to_group_bm
device_id,,,,
1002079943728939269,0,51,857,51.857
-1547860181818787117,1,51,860,51.860


In [20]:
gatrain.to_group.unique()

array(['M.35.51.843', 'M.30.51.865', 'M.30.51.847', ..., 'F.21.18.493',
       'M.38.101.1167', 'M.25.10.196'], dtype=object)

In [13]:
gatrain.to_group_bm.unique().shape

(1486,)

In [12]:
gatest.to_group_bm.unique().shape

(1589,)

In [14]:
len(set(gatrain.to_group_bm.unique())&set(gatest.to_group_bm.unique()))

1408

In [15]:
grpp_bm_unique = list(set(gatrain.to_group_bm.unique())&set(gatest.to_group_bm.unique()))

In [28]:
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

final_te_gender = pd.Series('X',index=gatest.index)
final_te_age = pd.Series(0,index=gatest.index)

for grpp_bm in grpp_bm_unique:
    df_tr_grpp_bm = tr_groupby_bm.get_group(grpp_bm)
    df_te_grpp_bm = te_groupby_bm.get_group(grpp_bm)
    tr_groupby_all = df_tr_grpp_bm.groupby('to_group')
    for grpp_all in df_tr_grpp_bm.to_group.unique():
        df_tr_grpp_all = tr_groupby_all.get_group(grpp_all)
        b = consecutive(df_tr_grpp_all.row)
        tr_consec = []
        for a in b:
            if a.shape[0]>1:
                tr_consec.append(a)
        for i in range(len(tr_consec)):
            df_tr_smallest = df_tr_grpp_all.ix[tr_consec[i].index]
            meann = df_tr_smallest.nid_order.mean()
            gndr = df_tr_smallest.gender.unique()[0]
            agee = df_tr_smallest.age.unique()[0]
            l_cnd = meann - 0.001
            h_cnd = meann + 0.001
            df_te_small = df_te_grpp_bm[(df_te_grpp_bm.nid_order>l_cnd)&(df_te_grpp_bm.nid_order<h_cnd)].sort('row')
            g = consecutive(df_te_small.row)
            te_consec = []
            for f in g:
                if f.shape[0]>1:
                    te_consec.append(f)
            minn = []
            for j in range(len(te_consec)):
                df_te_smallest = df_te_small.ix[te_consec[i].index]
                meann_te = df_te_smallest.nid_order.mean()
                minn.append(meann_te)
            indd = minn.index(min(minn))
            df_te_smallest_final = df_te_small.ix[te_consec[indd].index]
            final_te_gender[df_te_smallest_final.index] = gndr
            final_te_age[df_te_smallest_final.index] = agee

TypeError: 'int' object is not iterable